In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import pandas as pd

In [ ]:
dir(ds)

In [ ]:
import tsdm

ds = tsdm.datasets.InSilicoData()

In [ ]:
import tsdm

In [ ]:
task = tsdm.tasks.MIMIC_IV_Bilos2021()

In [ ]:
A

In [ ]:
A.loc[0]

In [ ]:
# A = task.split_idx_sparse.T.copy()

In [ ]:
pd.concat([A], keys=[0])

In [ ]:
idx = A.index
mask = idx.str.lower().isin(train_patterns)
value = idx[mask]
pd.Series(value.item(), index=idx)

In [ ]:
pd.Index(task.split_idx_sparse.T.index.to_series().to_dict())

In [ ]:
pd.Index(idx.to_series().to_dict().keys())

In [ ]:
A = A.loc[0]
idx = A.index
names = idx.names
*fold, partition = names
train_patterns = ["train", "training"]

In [ ]:
idx = idx.to_frame(index=False)
idx.partition = idx.partition.str.upper()
idx = idx.set_index(names).index

In [ ]:
T = idx.to_frame()
mask = T[partition].str.lower().isin(train_patterns)
train_folds = T[mask].copy()
train_folds = train_folds.drop(columns=names)
train_folds["key"] = list(T[mask].index)
if train_folds.index.nlevels > 1:
    train_folds = train_folds.droplevel(-1)
T = T.drop(columns=names)
T = train_folds.join(T, how="left")["key"]

In [ ]:
result = T.to_dict()

In [ ]:
?torch.nn.Module.forward

In [ ]:
import tsdm
from tsdm.utils.decorators import timefun

In [ ]:
ds = tsdm.datasets.InSilicoData(initialize=False);

In [ ]:
_, pre_cache_time = timefun(lambda: ds.dataset, append=True)()

In [ ]:
ds = tsdm.datasets.Electricity().dataset

In [ ]:
%time
ds = tsdm.datasets.Electricity(initialize=False);

In [ ]:
from typing import NamedTuple

In [ ]:
class Foo(NamedTuple):
    a: float
    b: int


tuple([1, 2, 3])

In [ ]:
a = 1

In [ ]:
import encoders

In [ ]:
import sys

sys.path

In [ ]:
import tsdm

In [ ]:
encoders.__path__

In [ ]:
import tsdm

In [ ]:
dir(tsdm)

In [ ]:
import tsdm
import pandas as pd

In [ ]:
DS = tsdm.datasets.USHCN_SmallChunkedSporadic()

In [ ]:
DS.dataset.sort_index(level="Time")

In [ ]:
dtypes = {
    "ID": "int16",
    "Time": "float32",
    "Value_0": "float32",
    "Value_1": "float32",
    "Value_2": "float32",
    "Value_3": "float32",
    "Value_4": "float32",
    "Mask_0": "bool",
    "Mask_1": "bool",
    "Mask_2": "bool",
    "Mask_3": "bool",
    "Mask_4": "bool",
}
df = pd.read_csv(PTH, dtype=dtypes)
df = pd.DataFrame(df)

In [ ]:
l = []
for k in range(5):
    key = f"CH_{k}"
    l.append(key)
    df[key] = df[f"Value_{k}"].where(df[f"Mask_{k}"])

df = df[["ID", "Time", *l]].sort_values(["ID", "Time"])

In [ ]:
df = df.set_index(["ID", "Time"])

In [ ]:
df.dtypes

In [ ]:
ts.Value_0.where(ts.Mask_0)

In [ ]:
ts["ID"].nunique()

In [ ]:
ds = tsdm.datasets.USHCN()

In [ ]:
from tsdm.encoders import TripletDecoder

In [ ]:
ts = ds.us_daily[["VALUE", "ELEMENT"]]

In [ ]:
enc = TripletDecoder(value_name="VALUE", var_name="ELEMENT")
enc.fit(ts)

In [ ]:
import modin.pandas as mpd

In [ ]:
ts = mpd.DataFrame(ts)

In [ ]:
s = ts[:1000].reset_index()

In [ ]:
type(ts)

In [ ]:
ts.reset_index().groupby(["COOP_ID", "time", "ELEMENT"])["VALUE"].aggregate(
    "mean"
).unstack()

In [ ]:
l = []

for value in ts.ELEMENT.unique():
    print(value)
    mask = ts.ELEMENT == value
    l.append(ts.VALUE[mask])

In [ ]:
ts.pivot(columns="ELEMENT", values="VALUE")